In [2]:
import os
os.environ["REDIS_URL"] = "redis://localhost:6379/0"
from shared_utils.clients.redis_client import redis_client

In [8]:
import uuid
from pydantic import BaseModel

class TestMessage(BaseModel):
    id: str
    value: str

stream_name = f"test-stream-{uuid.uuid4()}"
group_id = "test-group"
event = TestMessage(id="1", value="test-1")

print(stream_name, group_id, event, sep="\n")

test-stream-d13e4907-87da-4caf-ac12-5111ffff2cfa
test-group
id='1' value='test-1'


In [ ]:
# create a new consumer thread for the stream
redis_client.create_consumer(stream_name, group_id, max_concurrent_tasks=1)

2025-07-19 13:21:31,002 [RedisConsumer-test-stream-d13e4907-87da-4caf-ac12-5111ffff2cfa] WARNING: Running non-async callback <function test_callback at 0x7f25ab76ba60> in thread pool
/usr/lib/python3.12/asyncio/base_events.py:2000: RuntimeWarning: coroutine 'test_callback' was never awaited
  handle = None  # Needed to break cycles when an exception occurs.


In [ ]:
# register the callback to the consumer thread

from shared_utils import deserialize_event
import asyncio
@deserialize_event(TestMessage)
async def test_callback(event: TestMessage):
    print(f"Processing message {event.id} with value: {event.value}")
    await asyncio.sleep(1)
    print(f"Finished processing message {event.id}")

redis_client.register_callback(stream=stream_name, group_id=group_id, callback=test_callback)

In [12]:
# send a message to the stream

msg = event
redis_client.send(stream=stream_name, value=msg)